#  Chapter4

## 4.4 神经网络的进一步优化

In [1]:
import tensorflow as tf

##### 调整学习率——指数衰减法：tf.train.exponential_decay函数

1）等价于：decayed_learning_rate = learning_rate * decay_rate ^(global_step/decay_steps)

2）参数：staircase

staircase = False : 连续指数衰减学习率，不同的训练数据有不同的学习率

staircase = True : 阶梯状学习率（global_step/decay_step会被转化为整数），在这样的设置下，decay_steps通常代表了完整的使用一遍训练数据所需要的迭代轮数。这个迭代轮数也就是总训练样本数除以一盒batch中的训练样本数。这种设置的常用场景是每完整地过完一遍训练数据，学习率就减少一次。这可以使得训练数据集中的所有数据对模型训练有相等的作用。

3）tf.train.exponential_decay使用方法：

tf.train.exponential_decay（初始学习率, global_step, 每训练多少轮学习率衰减, 衰减系数, staircase）

In [ ]:
global_step = tf.Variable(0)

#通过exponential_decay函数生成学习率
learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.96, staircase = True)

#使用指数衰减的学习率，在minimize函数中传入global_step将自动更新
learning_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(...my loss..., global_step = global_step)

##### 过拟合问题——正则化

1）l2正则化——tf.contrib.layers.l2_regularizer / l1_regularizer

带l2正则化的顺势函数定义方法：

In [ ]:
w = tf.Variable(tf.random_normal([2,1], stddev = 1, seed = 1))
y = tf.matmul(x, w)

loss = tf.reduce_mean(tf.squre(y_- y)) + tf.contrib.layers.l2_regularizer(lambda)(w)

tf.contrib.layers.l2_regularizer / l1_regularizer的使用方法：

In [ ]:
weights = tf.constant([[1, -2],[-3, 4]])
with tf.Session() as sess:
    #输出为（|1|+|-2|+|-3|+|4|）*0.5 = 5，其中0.5是正则化项的权重
    print(sess.run(tf.contrib.layerss.l1_regularizer(.5)(weights)))
    #输出为（1^2+（-2）^2+(-3)^2+4^2）/2*0.5=7.5
    print(sess.run(tf.contrib.layers.l2_regularizer(.5)(weights)))

##### 在测试数据上更健壮的一种方法——滑动平均模型：tf.train.ExponentialMovingAverage



In [ ]:
v1 = tf.Variable(0,dtype = tf.float32)
#这里step变量模拟神经网络中迭代的轮数，可以用于动态控制衰减率
step = tf.Variable(0, trainable = False)

#定义一个滑动平均的类（class）.初始化时给定了衰减率（0.99）和控制衰减率的变量step
ema = tf.train.ExponentialMovingAverage(0.99, step)
#
maintain_average_op = ema.apply([v1])

with tf.Session() as sess:
    #初始化所有变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    #通过ema.average(v1)获取滑动平均之后变量的取值。在初始化之后变量v1的值和v1的值和v1的滑动平均都为0
    print sess.run([v1, ema.average(v1)])
    sess.run(tf.assign(v1,5))
    sess.run(maintain_average_op)
    print sess,run([v1,ema.average(v1)])
    
    #更新step的值为10000
    sess.run(tf.assign(step,10000))
    sess.run(tf.assign(v1,10))
    sess.run(maintain_average_op)
    print sess.run([v1,ema.average(v1)])
    
    sess.run(maintain_averages_op)
    print sess.run([v1,ema.average(v1)])